<a href="https://colab.research.google.com/github/sdikici/Demand_Forecasting_Prophet_DeepAR/blob/main/Data_Integration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d albertovidalrod/electricity-consumption-uk-20092022
! unzip electricity-consumption-uk-20092022.zip

 72% 14.0M/19.5M [00:00<00:00, 54.3MB/s]
100% 19.5M/19.5M [00:00<00:00, 53.8MB/s]
Archive:  electricity-consumption-uk-20092022.zip
  inflating: historic_demand_2009_2024.csv  
  inflating: historic_demand_2009_2024_noNaN.csv  
  inflating: historic_demand_year_2009.csv  
  inflating: historic_demand_year_2010.csv  
  inflating: historic_demand_year_2011.csv  
  inflating: historic_demand_year_2012.csv  
  inflating: historic_demand_year_2013.csv  
  inflating: historic_demand_year_2014.csv  
  inflating: historic_demand_year_2015.csv  
  inflating: historic_demand_year_2016.csv  
  inflating: historic_demand_year_2017.csv  
  inflating: historic_demand_year_2018.csv  
  inflating: historic_demand_year_2019.csv  
  inflating: historic_demand_year_2020.csv  
  inflating: historic_demand_year_2021.csv  
  inflating: historic_demand_year_2022.csv  
  inflating: historic_demand_year_2023.csv  
  inflating: historic_demand_year_2024.csv  


In [ ]:
!kaggle datasets download -d sercandikici/weatherdata2
! unzip weatherdata2.zip

  0% 0.00/679k [00:00<?, ?B/s]
100% 679k/679k [00:00<00:00, 76.2MB/s]
Archive:  weatherdata2.zip
  inflating: weather 2021-01-01 to 2024-01-01.csv  


In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
df_raw = pd.read_csv("historic_demand_2009_2024_noNaN.csv")
df_raw

,settlement_date,settlement_period,period_hour,nd,tsd,england_wales_demand,embedded_wind_generation,embedded_wind_capacity,embedded_solar_generation,embedded_solar_capacity,non_bm_stor,pump_storage_pumping,ifa_flow,ifa2_flow,britned_flow,moyle_flow,east_west_flow,nemo_flow,is_holiday
0,2009-01-01 00:00:00,1,0:00:00,37910,38704,33939,54,1403,0,0,0,33,2002,0,0,-161,0,0,1
1,2009-01-01 00:30:00,2,0:30:00,38047,38964,34072,53,1403,0,0,0,157,2002,0,0,-160,0,0,1
2,2009-01-01 01:00:00,3,1:00:00,37380,38651,33615,53,1403,0,0,0,511,2002,0,0,-160,0,0,1
3,2009-01-01 01:30:00,4,1:30:00,36426,37775,32526,50,1403,0,0,0,589,1772,0,0,-160,0,0,1
4,2009-01-01 02:00:00,5,2:00:00,35687,37298,31877,50,1403,0,0,0,851,1753,0,0,-160,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266747,2024-04-04 21:30:00,44,21:30:00,28724,29227,26137,2386,6562,0,15905,0,3,1004,992,1003,35,62,999,0
266748,2024-04-04 22:00:00,45,22:00:00,26996,27561,24603,2524,6562,0,15905,0,2,1004,992,981,-63,41,999,0
266749,2024-04-04 22:30:00,46,22:30:00,25460,26300,23265,2662,6562,0,15905,0,3,1007,991,979,-214,-123,999,0
266750,2024-04-04 23:00:00,47,23:00:00,23573,24586,21599,2866,6562,0,15905,0,5,1453,992,492,-236,-171,789,0


In [ ]:
x = df_raw[["settlement_date",	"period_hour",	"nd",	"tsd",	"england_wales_demand", "is_holiday"]][(df_raw["settlement_date"] >= "2021-01-01 00:00:00")&(df_raw["settlement_date"] <= "2024-01-01 00:00:00")]


In [ ]:
x.reset_index(inplace=True, drop=True)
x.head()

,settlement_date,period_hour,nd,tsd,england_wales_demand,is_holiday
0,2021-01-01 00:00:00,0:00:00,28354,28969,26130,1
1,2021-01-01 00:30:00,0:30:00,28501,29114,26281,1
2,2021-01-01 01:00:00,1:00:00,27759,28376,25557,1
3,2021-01-01 01:30:00,1:30:00,26912,27749,24792,1
4,2021-01-01 02:00:00,2:00:00,26004,27178,23933,1


In [ ]:
x[["settlement_date",	"tsd", "is_holiday"]].describe()

,tsd,is_holiday
count,52555.00000,52555.000000
mean,29080.93620,0.024679
std,5721.64994,0.155146
min,16513.00000,0.000000
25%,24807.00000,0.000000
50%,28384.00000,0.000000
75%,32506.00000,0.000000
max,47128.00000,1.000000


In [ ]:
x[["settlement_date",	"tsd", "is_holiday"]].isnull().sum()

settlement_date    0
tsd                0
is_holiday         0
dtype: int64

In [ ]:
df_weather = pd.read_csv("weather 2021-01-01 to 2024-01-01.csv")
df_weather['datetime'] = df_weather['datetime'].str.replace('T', ' ')
df_weather.head()

,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,london,2021-01-01 00:00:00,1.6,0.2,-0.4,86.63,0.0,0,NaN,0.0,...,1009.3,100.0,5.4,0.0,0.0,0,NaN,Overcast,cloudy,"03769099999,03680099999,03672099999,0378109999..."
1,london,2021-01-01 01:00:00,1.5,-0.4,-0.4,87.25,0.0,0,NaN,0.0,...,1009.2,100.0,3.8,0.0,0.0,0,NaN,Overcast,cloudy,"03769099999,03680099999,03672099999,0378109999..."
2,london,2021-01-01 02:00:00,1.6,-0.1,-0.4,86.63,0.0,0,NaN,0.0,...,1009.5,97.2,4.6,0.0,0.0,0,NaN,Overcast,cloudy,"03769099999,03680099999,03672099999,0378109999..."
3,london,2021-01-01 03:00:00,1.6,-0.8,-0.5,86.01,0.0,0,NaN,0.0,...,1010.1,93.0,4.7,0.0,0.0,0,NaN,Overcast,cloudy,"03769099999,03680099999,03672099999,0378109999..."
4,london,2021-01-01 04:00:00,1.0,-1.4,-0.9,87.20,0.0,0,NaN,0.0,...,1010.1,100.0,3.5,0.0,0.0,0,NaN,Overcast,cloudy,"03769099999,03680099999,03672099999,0378109999..."


In [ ]:
y= df_weather[["datetime","temp","feelslike","humidity"]]

In [ ]:
y[["datetime","temp"]].describe()

,temp
count,26304.000000
mean,12.426426
std,6.143512
min,-4.200000
25%,8.100000
50%,12.100000
75%,16.600000
max,39.800000


In [ ]:
y[["datetime","temp"]].isnull().sum()

datetime    0
temp        0
dtype: int64

In [ ]:
df3_year_energy = df_raw[["settlement_date","tsd","is_holiday"]]
df3_year_energy = df3_year_energy[(df3_year_energy["settlement_date"] >= "2021-01-01 00:00:00")&(df3_year_energy["settlement_date"] <= "2024-01-01 00:00:00")]
df3_year_energy.reset_index(inplace=True)
df3_year_energy

,index,settlement_date,tsd,is_holiday
0,209640,2021-01-01 00:00:00,28969,1
1,209641,2021-01-01 00:30:00,29114,1
2,209642,2021-01-01 01:00:00,28376,1
3,209643,2021-01-01 01:30:00,27749,1
4,209644,2021-01-01 02:00:00,27178,1
...,...,...,...,...
52550,262190,2023-12-31 22:00:00,24977,0
52551,262191,2023-12-31 22:30:00,24134,0
52552,262192,2023-12-31 23:00:00,23714,0
52553,262193,2023-12-31 23:30:00,23732,0


In [ ]:
#floor energy data, make it 2 hourly
df3_year_energy['settlement_date'] = pd.to_datetime(df3_year_energy['settlement_date'])
df3_year_energy['settlement_date'] = df3_year_energy['settlement_date'].dt.floor('2H')
df3_year_energy

,index,settlement_date,tsd,is_holiday
0,209640,2021-01-01 00:00:00,28969,1
1,209641,2021-01-01 00:00:00,29114,1
2,209642,2021-01-01 00:00:00,28376,1
3,209643,2021-01-01 00:00:00,27749,1
4,209644,2021-01-01 02:00:00,27178,1
...,...,...,...,...
52550,262190,2023-12-31 22:00:00,24977,0
52551,262191,2023-12-31 22:00:00,24134,0
52552,262192,2023-12-31 22:00:00,23714,0
52553,262193,2023-12-31 22:00:00,23732,0


In [ ]:
df_2hourly_energy = df3_year_energy.groupby('settlement_date').agg({'tsd': 'mean','is_holiday': 'mean'}).reset_index()
df_2hourly_energy

,settlement_date,tsd,is_holiday
0,2021-01-01 00:00:00,28552.00,1.0
1,2021-01-01 02:00:00,26579.75,1.0
2,2021-01-01 04:00:00,25236.00,1.0
3,2021-01-01 06:00:00,25792.25,1.0
4,2021-01-01 08:00:00,28162.75,1.0
...,...,...,...
13136,2023-12-31 16:00:00,34006.00,0.0
13137,2023-12-31 18:00:00,32273.00,0.0
13138,2023-12-31 20:00:00,27456.50,0.0
13139,2023-12-31 22:00:00,24139.25,0.0


prepare weather data

In [ ]:
weather3_year = df_weather[["datetime","temp"]]
weather3_year = weather3_year[(weather3_year["datetime"] >= "2021-01-01 00:00:00")&(weather3_year["datetime"] <= "2024-01-01 00:00:001")]
weather3_year["datetime"] = pd.to_datetime(weather3_year["datetime"])
weather3_year

,datetime,temp
0,2021-01-01 00:00:00,1.6
1,2021-01-01 01:00:00,1.5
2,2021-01-01 02:00:00,1.6
3,2021-01-01 03:00:00,1.6
4,2021-01-01 04:00:00,1.0
...,...,...
26276,2023-12-31 20:00:00,8.3
26277,2023-12-31 21:00:00,8.6
26278,2023-12-31 22:00:00,8.1
26279,2023-12-31 23:00:00,8.6


In [ ]:
weather3_year["datetime"] = pd.to_datetime(weather3_year["datetime"])
weather3_year["datetime"] = weather3_year['datetime'].dt.floor('2H')
weather3_year

,datetime,temp
0,2021-01-01 00:00:00,1.6
1,2021-01-01 00:00:00,1.5
2,2021-01-01 02:00:00,1.6
3,2021-01-01 02:00:00,1.6
4,2021-01-01 04:00:00,1.0
...,...,...
26276,2023-12-31 20:00:00,8.3
26277,2023-12-31 20:00:00,8.6
26278,2023-12-31 22:00:00,8.1
26279,2023-12-31 22:00:00,8.6


In [ ]:
df_2hourly_weather = weather3_year.groupby('datetime').agg({'temp': 'mean'}).reset_index()
df_2hourly_weather,df_2hourly_energy

(                 datetime  temp
 0     2021-01-01 00:00:00  1.55
 1     2021-01-01 02:00:00  1.60
 2     2021-01-01 04:00:00  0.55
 3     2021-01-01 06:00:00 -0.35
 4     2021-01-01 08:00:00 -0.10
 ...                   ...   ...
 13136 2023-12-31 16:00:00  7.60
 13137 2023-12-31 18:00:00  8.20
 13138 2023-12-31 20:00:00  8.45
 13139 2023-12-31 22:00:00  8.35
 13140 2024-01-01 00:00:00  8.60
 
 [13141 rows x 2 columns],
           settlement_date         tsd  is_holiday
 0     2021-01-01 00:00:00  168.973371         1.0
 1     2021-01-01 02:00:00  163.032972         1.0
 2     2021-01-01 04:00:00  158.858428         1.0
 3     2021-01-01 06:00:00  160.599658         1.0
 4     2021-01-01 08:00:00  167.817609         1.0
 ...                   ...         ...         ...
 13136 2023-12-31 16:00:00  184.407158         0.0
 13137 2023-12-31 18:00:00  179.646876         0.0
 13138 2023-12-31 20:00:00  165.700030         0.0
 13139 2023-12-31 22:00:00  155.368111         0.0
 13140 2024-01

In [ ]:
df_2hourly_energy['settlement_date_rounded'] = df_2hourly_energy['settlement_date'].dt.round('2H')
df_2hourly_weather['datetime_rounded'] = df_2hourly_weather['datetime'].dt.round('2H')

In [ ]:
merged_df = pd.merge(df_2hourly_energy, df_2hourly_weather, left_on='settlement_date_rounded', right_on='datetime_rounded', how='inner')
merged_df = merged_df.drop(columns=['datetime','settlement_date_rounded', 'datetime_rounded'])
merged_df.head()

,settlement_date,tsd,is_holiday,temp
0,2021-01-01 00:00:00,168.973371,1.0,1.55
1,2021-01-01 02:00:00,163.032972,1.0,1.60
2,2021-01-01 04:00:00,158.858428,1.0,0.55
3,2021-01-01 06:00:00,160.599658,1.0,-0.35
4,2021-01-01 08:00:00,167.817609,1.0,-0.10


In [ ]:
merged_df.tail()

,settlement_date,tsd,is_holiday,temp
13136,2023-12-31 16:00:00,184.407158,0.0,7.60
13137,2023-12-31 18:00:00,179.646876,0.0,8.20
13138,2023-12-31 20:00:00,165.700030,0.0,8.45
13139,2023-12-31 22:00:00,155.368111,0.0,8.35
13140,2024-01-01 00:00:00,153.186161,1.0,8.60


In [ ]:
merged_df.to_csv('merged_data.csv', index=False)
from google.colab import files
files.download('merged_data.csv')